In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd 
import re

In [2]:
url = "https://www.mushroomexpert.com/"
response= requests.get(url)
soup = BeautifulSoup(response.content, "html.parser")
table = soup.find(lambda tag: tag.name == 'table' and tag.find('i') and tag.find('i').string == 'Clavulinopsis')

In [3]:
species=table.find_all('a')
number_of_species = len(species)
print("Number of species is:", number_of_species)

Number of species is: 4


In [4]:
df=pd.DataFrame(columns=['SpeciesName','Ecology','Odor and Taste'])

In [11]:

for specie in species:
    result=[]
    url = 'https://www.mushroomexpert.com/'+specie['href']
    response=requests.get(url)
    soup=BeautifulSoup(response.content,'html.parser')
    
    name= soup.find('font', {'size':'+0'}).text
    result.append(name)

    wanted_tag = soup.find(lambda tag: tag.name == 'p' and re.search('saprobic|mycorrhizal|parasitic', tag.text, re.IGNORECASE))
    ecology=wanted_tag.a.text
    result.append(ecology)

    wanted_tag= soup.find(lambda tag: tag.name == 'p' and 'Odor and Taste' in tag.text)
    odor_taste= wanted_tag.text[16:]
    result.append(odor_taste)

    df.loc[len(df)]=result

df.to_csv('Clavulinopsis(MushroomExpert).csv',index=False)